# Generating CellSpace for GML

In [19]:
# Re-defining the original list and adjusting the code to remove all double quotes from the second, third, and fourth elements

original_list = [
    'C1,N1,1,Room 1,"0,0,0","100,0,0","100,100,0","0,100,0","0,0,0"',
    '"0,0,100","100,0,100","100,100,100","0,100,100","0,0,100"',
    '"0,0,0","100,0,0","100,0,100","0,0,100","0,0,0","100,0,0","100,100,0","100,100,100","100,0,100","100,0,0","100,100,0","0,100,0","0,100,100","100,100,100","100,100,0","0,100,0","0,0,0","0,0,100","0,100,100","0,100,0"'
]

def reformat_list_remove_quotes(lst):
    # Extract room description from the first element
    room_description = lst[0].split(',')[0:4]
    room_description = ','.join(room_description)

    # Prepare the new list with the room description as the first element
    new_list = [room_description]

    # Remove double quotes for the coordinates in the first element and append directly
    coordinates_raw = lst[0].split(',')[4:]  # Get the part of the string with coordinates
    coordinates_str = ','.join(coordinates_raw).replace('"', '')  # Remove all double quotes
    new_list.append(coordinates_str)

    # Append the other elements without quotes
    for item in lst[1:]:
        new_list.append(item.replace('"', ''))

    return new_list

# Reformatting the list to remove all double quotes from the specified elements
correctly_formatted_list_no_quotes = reformat_list_remove_quotes(original_list)

# Printing the list after removing double quotes
for item in correctly_formatted_list_no_quotes:
    print(item)
    print(type(item))


C1,N1,1,Room 1
<class 'str'>
0,0,0,100,0,0,100,100,0,0,100,0,0,0,0
<class 'str'>
0,0,100,100,0,100,100,100,100,0,100,100,0,0,100
<class 'str'>
0,0,0,100,0,0,100,0,100,0,0,100,0,0,0,100,0,0,100,100,0,100,100,100,100,0,100,100,0,0,100,100,0,0,100,0,0,100,100,100,100,100,100,100,0,0,100,0,0,0,0,0,0,100,0,100,100,0,100,0
<class 'str'>


In [20]:
def get_list_from_csv(f_path):
    final_data = []
    with open(f_path, "r") as f:
        for line in f.readlines():
            line = line.strip('\n')
            split_line = line.split('ADD')
            processed_line = [element.lstrip(',').rstrip(',') for element in split_line]
            split_line = reformat_list_remove_quotes(processed_line)
            final_data.append(split_line)
        return final_data
file_path = "C:/Users/SID-DRW/OneDrive - Delft University of Technology/Desktop/new_boundaries_0204/hugo_test/cellspace_backup.csv"
new_data_list = get_list_from_csv(file_path)    
print(new_data_list)   

[['C1,N1,100,Room1', '0,0,0,100,0,0,100,100,0,0,100,0,0,0,0', '0,0,100,100,0,100,100,100,100,0,100,100,0,0,100', '0,0,0,100,0,0,100,0,100,0,0,100,0,0,0,100,0,0,100,100,0,100,100,100,100,0,100,100,0,0,100,100,0,0,100,0,0,100,100,100,100,100,100,100,0,0,100,0,0,0,0,0,0,100,0,100,100,0,100,0'], ['C2,N2,100,Room2', '100,0,0,200,0,0,200,100,0,100,100,0,100,0,0', '100,0,100,200,0,100,200,100,100,100,100,100,100,0,100', '100,0,0,200,0,0,200,0,100,100,0,100,100,0,0,200,0,0,200,100,0,200,100,100,200,0,100,200,0,0,200,100,0,100,100,0,100,100,100,200,100,100,200,100,0,100,100,0,100,0,0,100,0,100,100,100,100,100,100,0']]


In [21]:

def generate_primal_space_features_corrected(data_list):
    gml_text = 'xsi:schemaLocation="http://www.opengis.net/indoorgml/1.0/core http://schemas.opengis.net/indoorgml/1.0/indoorgmlcore.xsd http://www.opengis.net/indoorgml/1.0/navigation http://schemas.opengis.net/indoorgml/1.0/indoorgmlnavi.xsd">\n'
    gml_text += '  <gml:boundedBy xsi:nil="true"/>\n'
    gml_text += "  <primalSpaceFeatures>\n"
    gml_text += '    <PrimalSpaceFeatures gml:id="PS1">\n'
    gml_text += '      <gml:boundedBy xsi:nil="true"/>\n'
    for index, data in enumerate(data_list):
        header = data[0].split(',')
        gml_id, xlink_href, area, name = header[0], header[1], header[2], header[3]

        gml_text += f"""      <cellSpaceMember>
        <CellSpace gml:id="{gml_id}">
          <gml:name>{name}</gml:name>
          <gml:area>{area}</gml:area>
          <gml:boundedBy xsi:nil="true"/>
          <cellSpaceGeometry>
            <Geometry3D>
              <gml:Solid gml:id="CG-{gml_id}">  
                <gml:exterior>
                  <gml:Shell>\n"""

        # Process the first two coordinate strings normally
        for coords_str in data[1:3]:
            coords = coords_str.split(',')
            gml_text += '                    <gml:surfaceMember>\n'
            gml_text += '                      <gml:Polygon>\n'
            gml_text += '                        <gml:exterior>\n'
            gml_text += '                          <gml:LinearRing>\n'
            for i in range(0, len(coords), 3):
                x, y, z = coords[i:i+3]
                gml_text += f'                            <gml:pos srsDimension="3">{float(x):.2f} {float(y):.2f} {float(z):.2f}</gml:pos>\n'
            gml_text += '                          </gml:LinearRing>\n'
            gml_text += '                        </gml:exterior>\n'
            gml_text += '                      </gml:Polygon>\n'
            gml_text += '                    </gml:surfaceMember>\n'

        # Special handling for the third group to distribute into the 3rd, 4th, 5th, and 6th groups
        third_group_coords = data[3].split(',')
        for i in range(0, len(third_group_coords), 15):  # Process 5 points at a time
            gml_text += '                    <gml:surfaceMember>\n'
            gml_text += '                      <gml:Polygon>\n'
            gml_text += '                        <gml:exterior>\n'
            gml_text += '                          <gml:LinearRing>\n'
            for j in range(i, i + 15, 3):
                if j < len(third_group_coords):  # Prevent index out of bounds
                    x, y, z = third_group_coords[j:j+3]
                    gml_text += f'                            <gml:pos srsDimension="3">{float(x):.2f} {float(y):.2f} {float(z):.2f}</gml:pos>\n'
            gml_text += '                          </gml:LinearRing>\n'
            gml_text += '                        </gml:exterior>\n'
            gml_text += '                      </gml:Polygon>\n'
            gml_text += '                    </gml:surfaceMember>\n'

        gml_text += f"""                  </gml:Shell>
                </gml:exterior>
              </gml:Solid>
            </Geometry3D>   
          </cellSpaceGeometry>
          <duality xlink:href="#{xlink_href}"/>
        </CellSpace>
      </cellSpaceMember>"""

        if index < len(data_list) - 1:  # Add a newline before the next cellSpaceMember for all but the last one
            gml_text += "\n"

    #gml_text += """
    #</PrimalSpaceFeatures>
  #</primalSpaceFeatures>"""

    return gml_text.strip()

# Generate the primal space features text with corrected formatting
data_list = [
    ['C1,N1,100,Room1', '0,0,0,100,0,0,100,100,0,0,100,0,0,0,0', '0,0,100,100,0,100,100,100,100,0,100,100,0,0,100', '0,0,0,100,0,0,100,0,100,0,0,100,0,0,0,100,0,0,100,100,0,100,100,100,100,0,100,100,0,0,100,100,0,0,100,0,0,100,100,100,100,100,100,100,0,0,100,0,0,0,0,0,0,100,0,100,100,0,100,0'],
    ['C2,N2,100,Room2', '100,0,0,200,0,0,200,100,0,100,100,0,100,0,0', '100,0,100,200,0,100,200,100,100,100,100,100,100,0,100', '100,0,0,200,0,0,200,0,100,100,0,100,100,0,0,200,0,0,200,100,0,200,100,100,200,0,100,200,0,0,200,100,0,100,100,0,100,100,100,200,100,100,200,100,0,100,100,0,100,0,0,100,0,100,100,100,100,100,100,0']
] 
primal_space_features_text = generate_primal_space_features_corrected(new_data_list)
print(primal_space_features_text)


xsi:schemaLocation="http://www.opengis.net/indoorgml/1.0/core http://schemas.opengis.net/indoorgml/1.0/indoorgmlcore.xsd http://www.opengis.net/indoorgml/1.0/navigation http://schemas.opengis.net/indoorgml/1.0/indoorgmlnavi.xsd">
  <gml:boundedBy xsi:nil="true"/>
  <primalSpaceFeatures>
    <PrimalSpaceFeatures gml:id="PS1">
      <gml:boundedBy xsi:nil="true"/>
      <cellSpaceMember>
        <CellSpace gml:id="C1">
          <gml:name>Room1</gml:name>
          <gml:area>100</gml:area>
          <gml:boundedBy xsi:nil="true"/>
          <cellSpaceGeometry>
            <Geometry3D>
              <gml:Solid gml:id="CG-C1">  
                <gml:exterior>
                  <gml:Shell>
                    <gml:surfaceMember>
                      <gml:Polygon>
                        <gml:exterior>
                          <gml:LinearRing>
                            <gml:pos srsDimension="3">0.00 0.00 0.00</gml:pos>
                            <gml:pos srsDimension="3">100.00 0.00 0.0

# Generating Cell Space Boundary for GML

In [22]:
def parse_doors_csv_without_quotes(file_path):
    doors_list = []
    try:
        with open(file_path, 'r') as file:
            
            for line in file:
                parts = line.strip().split(',')
                # Extract 'id' and 'connects', ensure no double quotes
                id, connects = parts[0].replace('"', ''), parts[1].replace('"', '')
                # Concatenate coordinates, explicitly removing double quotes
                coordinates = ','.join(part.replace('"', '') for part in parts[2:])
                doors_list.append([id, connects, coordinates])
    except Exception as e:
        print(f"Error reading file: {e}")
    return doors_list

# Assuming the file path
file_path = "C:/Users/SID-DRW/OneDrive - Delft University of Technology/Desktop/new_boundaries_0204/hugo_test/doors.csv"
doors_list_manual = parse_doors_csv_without_quotes(file_path)
print(doors_list_manual)



[['SD1', 'B1', '100.00,30.00,0.00,100.00,60.00,0.00,100.00,60.00,50.00,100.00,30.00,50.00,100.00,30.00,0.000']]


In [23]:
list_of_lists = [['SD1','B1','100.00,30.00,0.00,100.00,60.00,0.00,100.00,60.00,50.00,100.00,30.00,50.00,100.00,30.00,0.000'],
                 ['SD2','B2','100.00,30.00,0.00,100.00,60.00,0.00,100.00,60.00,50.00,100.00,30.00,50.00,100.00,30.00,0.000']]

def create_xml_text(data_list):
    text_output = ""
    for item in data_list:
        duality, gml_id, coordinates = item
        coords = coordinates.split(',')
        text_output += f"""      <cellSpaceBoundaryMember>
        <CellSpaceBoundary gml:id="{gml_id}">
          <gml:boundedBy xsi:nil="true"/>
          <duality xlink:href="#{duality}"/>
          <cellSpaceBoundaryGeometry>
            <geometry3D>
              <gml:Polygon gml:id="CBG-{gml_id}">
                <gml:exterior>
                  <gml:LinearRing>\n"""
        for i in range(0, len(coords), 3):
            text_output += f"                    <gml:pos srsDimension=\"3\">{coords[i]} {coords[i+1]} {coords[i+2]}</gml:pos>\n"
        text_output += """                  </gml:LinearRing>
                </gml:exterior>
              </gml:Polygon>
            </geometry3D>
          </cellSpaceBoundaryGeometry>
        </CellSpaceBoundary>
      </cellSpaceBoundaryMember>\n"""
    text_output += "    </PrimalSpaceFeatures>\n"
    text_output += "  </primalSpaceFeatures>"
    return text_output

# This script should be executed in a suitable Python environment.
boundary_text = create_xml_text(doors_list_manual)
print(boundary_text)


      <cellSpaceBoundaryMember>
        <CellSpaceBoundary gml:id="B1">
          <gml:boundedBy xsi:nil="true"/>
          <duality xlink:href="#SD1"/>
          <cellSpaceBoundaryGeometry>
            <geometry3D>
              <gml:Polygon gml:id="CBG-B1">
                <gml:exterior>
                  <gml:LinearRing>
                    <gml:pos srsDimension="3">100.00 30.00 0.00</gml:pos>
                    <gml:pos srsDimension="3">100.00 60.00 0.00</gml:pos>
                    <gml:pos srsDimension="3">100.00 60.00 50.00</gml:pos>
                    <gml:pos srsDimension="3">100.00 30.00 50.00</gml:pos>
                    <gml:pos srsDimension="3">100.00 30.00 0.000</gml:pos>
                  </gml:LinearRing>
                </gml:exterior>
              </gml:Polygon>
            </geometry3D>
          </cellSpaceBoundaryGeometry>
        </CellSpaceBoundary>
      </cellSpaceBoundaryMember>
    </PrimalSpaceFeatures>
  </primalSpaceFeatures>


In [24]:
#第九行的“<gml:boundedBy xsi:nil="true"/>”应该缩进两格
title_text = '<IndoorFeatures\n'+'gml:id="IFt_s_76"\n'+'xmlns:gml="http://www.opengis.net/gml/3.2"\n'+'xmlns:xlink="http://www.w3.org/1999/xlink"\n'+ 'xmlns:core="http://www.opengis.net/indoorgml/1.0/core"\n'+'xmlns:navi="http://www.opengis.net/indoorgml/1.0/navigation"\n'+'xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"'
final_text = title_text + '\n'+primal_space_features_text +'\n'+ boundary_text

print(final_text)

<IndoorFeatures
gml:id="IFt_s_76"
xmlns:gml="http://www.opengis.net/gml/3.2"
xmlns:xlink="http://www.w3.org/1999/xlink"
xmlns:core="http://www.opengis.net/indoorgml/1.0/core"
xmlns:navi="http://www.opengis.net/indoorgml/1.0/navigation"
xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
xsi:schemaLocation="http://www.opengis.net/indoorgml/1.0/core http://schemas.opengis.net/indoorgml/1.0/indoorgmlcore.xsd http://www.opengis.net/indoorgml/1.0/navigation http://schemas.opengis.net/indoorgml/1.0/indoorgmlnavi.xsd">
  <gml:boundedBy xsi:nil="true"/>
  <primalSpaceFeatures>
    <PrimalSpaceFeatures gml:id="PS1">
      <gml:boundedBy xsi:nil="true"/>
      <cellSpaceMember>
        <CellSpace gml:id="C1">
          <gml:name>Room1</gml:name>
          <gml:area>100</gml:area>
          <gml:boundedBy xsi:nil="true"/>
          <cellSpaceGeometry>
            <Geometry3D>
              <gml:Solid gml:id="CG-C1">  
                <gml:exterior>
                  <gml:Shell>
                